In [ ]:
from fpdf import FPDF
import pandas as pd
import os
import geopandas as gpd
import matplotlib.pyplot as plt
from io import BytesIO

def _create_empty_pdf():
    pdf = FPDF()
    pdf.add_page()

    return pdf
    
def _add_title(pdf, row):
    pdf.set_font('helvetica', size=16, style='B')
    pdf.cell(200, 10, text=f"Dataset: {row['data_title'].values[0]}", new_x="LMARGIN", new_y="NEXT", align='C')

    return pdf

def _add_metadata_content(pdf, row):
    pdf = _add_text_box(pdf, "Dataset summary", row.data_summary[0].replace("\n", " "))
    pdf = _add_text_box(pdf, "Dataset publisher", row.publisher_fullname[0])
    pdf = _add_text_box(pdf, "High level source", row.source[0])
    pdf = _add_text_box(pdf, "Original format", row.format[0])
    pdf = _add_text_box(pdf, "Homeage link",  f"[{row.homepage[0]}]({row.homepage[0]})")
    pdf = _add_text_box(pdf, "Data download link",  f"[{row.data_link[0]}]({row.data_link[0]})")
    pdf = _add_text_box(pdf, "Metadata link",  f"[{row.metadata_link[0]}]({row.metadata_link[0]})")
    pdf = _add_text_box(pdf, "Data license/use constraints", row.use_constraints[0])

    return pdf

def _add_text_box(pdf, title, text):
    pdf.set_font('helvetica', size=12, style = 'B')
    pdf.multi_cell(200, 10, text = title,  new_x="LMARGIN", new_y="NEXT", align = 'L')
    pdf.set_font('helvetica', size=12)
    pdf.multi_cell(200, 10, text = text,  new_x="LMARGIN", new_y="NEXT", align = 'L', markdown=True)

    return pdf

def _add_data_content(pdf, gdf):
    _add_text_box(pdf, title="Dataset features", text = ', '.join(gdf.columns))
    _add_text_box(pdf, title="Number of rows", text = str(gdf.shape[0]))

    return pdf

def _add_map(pdf, gdf):
    pdf.set_font('helvetica', size=12, style = 'B')
    pdf.multi_cell(200, 10, text = "Plot of geometry feature",  new_x="LMARGIN", new_y="NEXT", align = 'L')

    plt.figure()  # Create a new figure object
    gdf.plot()

    img_buf = BytesIO()  # Create image object
    plt.savefig(img_buf, dpi=200)  # Save the image

    pdf.image(img_buf, w=pdf.epw)  # Make the image full width
    img_buf.close()

    return pdf


In [ ]:
os.chdir("/Users/canyonfoot/Documents/python_proj/nwf-process-geodata")
metadata_row = pd.read_csv("output_metadata.csv").head(1)

In [ ]:
str(metadata_row.columns)

In [ ]:
gdf = gpd.read_file("/Users/canyonfoot/Documents/python_proj/nwf-process-geodata/export_test/data/processed/wildlife/aquatic/crucial_stream_corridors.geojson")

In [ ]:
pdf = _create_empty_pdf()
pdf = _add_title(pdf, metadata_row)
pdf = _add_metadata_content(pdf, metadata_row)
pdf = _add_data_content(pdf, gdf)
pdf = _add_map(pdf, gdf)

In [ ]:
# Save
pdf.output("simple_demo.pdf")